In [ ]:
import numpy as np
import io
import pandas as pd
import os
import fnmatch
import shutil
import re
from ruamel.yaml import YAML

import xarray as xr
import h5py
from brainio.assemblies import NeuronRecordingAssembly

In [ ]:
############### Check Directories #############################################
###############################################################################

df = pd.read_excel( '/braintree/home/aliya277/dandi_brainscore/pico_inventory_new.xlsx'  )
SubjectName = 'pico'
storage_dir = '/braintree/home/aliya277/inventory_new'

i = 0
for imageset, excelname, date_, time_, hasspike, haspsth, hash5, spikepath, psthpath, h5path in zip(df['ImageSet'], df['(excel) Stimuli'], df['date'], df['time'], df['Has SpikeTime'], \
    df['Has psth'], df['Has h5'], df['Path: SpikeTimes'], df['Path: psth'], df['Path: h5']):
    date = f'20{date_}'

    if len(str(time_)) != 6: time = f'0{time_}'
    else: time = time_
    if imageset.startswith('facee'):
        correctdate = date
        print('path: ', h5path)
        # break
    i += 1
print(i)
        
for imageset, excelname, date_, time_, hasspike, haspsth, hash5, spikepath, psthpath, h5path in zip(df['ImageSet'], df['(excel) Stimuli'], df['date'], df['time'], df['Has SpikeTime'], \
    df['Has psth'], df['Has h5'], df['Path: SpikeTimes'], df['Path: psth'], df['Path: h5']):
    date = f'20{date_}'
    if len(str(time_)) != 6: time = f'0{time_}'
    else: time = time_
    if imageset.startswith('norm') and date == correctdate:
        # correctdate = date
        print('normpath: ',h5path)
        # break

path:  nan
path:  /braintree/data2/active/users/sgouldin/projects/faceemovids/monkeys/pico/h5/230908.pico.rsvp.faceemovids.experiment_psth_raw.h5
path:  /braintree/data2/active/users/sgouldin/projects/faceemovids/monkeys/pico/h5/230908.pico.rsvp.faceemovids.experiment_psth_raw.h5
normpath:  /braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/230908.pico.rsvp.normalizers.experiment_psth_raw.h5


In [2]:
h5_recording = h5py.File('/braintree/data2/active/users/sgouldin/projects/faceemovids/monkeys/pico/h5/230908.pico.rsvp.faceemovids.experiment_psth_raw.h5', 'r')
h5_norm      = h5py.File('/braintree/data2/active/users/sgouldin/projects/normalizers/monkeys/pico/h5/230908.pico.rsvp.normalizers.experiment_psth_raw.h5', 'r')

# image has been shown for 500 ms

In [10]:
data = h5_norm
psth = np.asarray(data['psth'])  # Shaped images x repetitions x time_bins x channels
meta = data['meta']
timebase = np.arange(meta['start_time_ms'][()], meta['stop_time_ms'][()], meta['tb_ms'][()])
timebins = np.asarray([[int(x), int(x)+int(meta['tb_ms'][()])] for x in timebase])
assert len(timebase) == psth.shape[2]

assembly = xr.DataArray(psth,
                coords={'repetition': ('repetition', list(range(psth.shape[1]))),
                        'time_bin_id': ('time_bin', list(range(psth.shape[2]))),
                        'time_bin_start': ('time_bin', [x[0] for x in timebins]),
                        'time_bin_stop': ('time_bin', [x[1] for x in timebins])},
                dims=['image', 'repetition', 'time_bin', 'neuroid'])

# Collapse dimensions 'image' and 'repetitions' into a single 'presentation' dimension
assembly = assembly.stack(presentation=('image', 'repetition')).reset_index('presentation')
assembly = assembly.drop('image')
assembly = NeuronRecordingAssembly(assembly)  # Convert to assembly

assembly.shape

(30, 192, 1806)

In [ ]:
assembly.isel(time_bin = slice(7, 17)).sum('time_bin')

In [15]:
from ndashboard.nquality.raw_data_template import SessionNeuralData

SessionNeuralData(assembly.isel(time_bin = slice(7, 17)).sum('time_bin'))

AssertionError: da_presentation.coords:Coordinates:
  * presentation  (presentation) int64 0 1 2 3 4 5 6 7 ... 14 15 16 17 18 19 20, Required coordinates: {'stimulus_id', 'unix_timestamp'}

In [11]:
assembly.isel(time_bin = slice(7, 17)).sum('time_bin')

(192, 1806)

In [9]:
assembly.time_bin

<xarray.NeuronRecordingAssembly 'time_bin' (time_bin: 30)>
array([(0, 0, 10), (1, 10, 20), (2, 20, 30), (3, 30, 40), (4, 40, 50),
       (5, 50, 60), (6, 60, 70), (7, 70, 80), (8, 80, 90), (9, 90, 100),
       (10, 100, 110), (11, 110, 120), (12, 120, 130), (13, 130, 140),
       (14, 140, 150), (15, 150, 160), (16, 160, 170), (17, 170, 180),
       (18, 180, 190), (19, 190, 200), (20, 200, 210), (21, 210, 220),
       (22, 220, 230), (23, 230, 240), (24, 240, 250), (25, 250, 260),
       (26, 260, 270), (27, 270, 280), (28, 280, 290), (29, 290, 300)],
      dtype=object)
Coordinates:
  * time_bin        (time_bin) MultiIndex
  - time_bin_id     (time_bin) int64 0 1 2 3 4 5 6 7 ... 22 23 24 25 26 27 28 29
  - time_bin_start  (time_bin) int64 0 10 20 30 40 50 ... 250 260 270 280 290
  - time_bin_stop   (time_bin) int64 10 20 30 40 50 60 ... 260 270 280 290 300

In [7]:
assembly.shape

(30, 192, 1806)

In [7]:
print(psth.shape)
data['meta']['start_time_ms'][()], data['meta']['stop_time_ms'][()], data['meta']['tb_ms'][()]

(86, 21, 30, 192)


(0.0, 300.0, 10.0)

In [8]:
len(timebins)

30

In [9]:
def load_responses(data_dir, stimuli, store_temporal=False):
    data_dir_h5 = data_dir / 'projects' / 'HVM-var6-subset-2023' / 'monkeys' / 'pico' / 'h5'
    assert os.path.isdir(data_dir_h5)
    data = h5py.File(data_dir_h5 / f'pico.rsvp.HVM-var6-subset-2023.experiment_psth.h5', 'r')
    assert np.all([x in data.keys() for x in ['psth', 'meta']])
    psth = np.asarray(data['psth'])  # Shaped images x repetitions x time_bins x channels
    meta = data['meta']
    print(psth.shape)
    #print(meta['start_time_ms'][()])
    timebase = np.arange(meta['start_time_ms'][()], meta['stop_time_ms'][()], meta['tb_ms'][()])
    timebins = np.asarray([[int(x), int(x)+int(meta['tb_ms'][()])] for x in timebase])
    assert len(timebase) == psth.shape[2]

    if not store_temporal:
        # Compute firing rate for given time bins
        new_timebins = np.asarray([[70, 170], [170, 270], [50, 100], [100, 150], [150, 200], [200, 250], [70, 270]])
        rate = np.empty((len(new_timebins), psth.shape[0], psth.shape[1], psth.shape[3]))
        for idx, tb in enumerate(new_timebins):
            t_cols = np.where((timebase >= tb[0]) & (timebase < tb[1]))[0]
            rate[idx] = np.nanmean(psth[:, :, t_cols, :], axis=2)  # Shaped time_bins x images x repetitions x channels
        psth = np.moveaxis(rate, 0, 2)  # Shaped images x repetitions x time_bins x channels
        timebins = new_timebins

    assembly = xr.DataArray(psth,
                            coords={'repetition': ('repetition', list(range(psth.shape[1]))),
                                    'time_bin_id': ('time_bin', list(range(psth.shape[2]))),
                                    'time_bin_start': ('time_bin', [x[0] for x in timebins]),
                                    'time_bin_stop': ('time_bin', [x[1] for x in timebins])},
                            dims=['image', 'repetition', 'time_bin', 'neuroid'])

    # Add neuroid related meta data
    neuroid_meta = pd.DataFrame(json.load(open(data_dir / 'array-metadata' / '021023_pico_mapping_noCIT_adapter_version.json')))
    for column_name, column_data in neuroid_meta.items():
        assembly = assembly.assign_coords(**{f'{column_name}': ('neuroid', list(column_data.values))})

    # Add stimulus related meta data
    for column_name, column_data in stimuli.items():
        assembly = assembly.assign_coords(**{f'{column_name}': ('image', list(column_data.values))})

    # Collapse dimensions 'image' and 'repetitions' into a single 'presentation' dimension
    assembly = assembly.stack(presentation=('image', 'repetition')).reset_index('presentation')
    assembly = assembly.drop('image')
    assembly = NeuronRecordingAssembly(assembly)  # Convert to assembly

    # Filter noisy electrodes
    data = h5py.File(data_dir_h5 / f'pico.rsvp.HVM-var6-subset-2023.normalizer_psth.h5')
    assert np.all([x in data.keys() for x in ['psth', 'meta']])
    psth = np.asarray(data['psth'])  # Shaped images x repetitions x time_bins x channels
    meta = data['meta']

    psth_index = np.arange(0,86)
    psth_index = np.where(psth_index != 26)
    psth = psth[psth_index, :, :, :]  # Remove responses to gray image (26th image)
    psth = psth.squeeze()

    timebase = np.arange(meta['start_time_ms'][()], meta['stop_time_ms'][()], meta['tb_ms'][()])
    t_cols = np.where((timebase >= 70) & (timebase < 170))[0]
    rate = np.mean(psth[:, :, t_cols, :], axis=2)
    #print(rate.shape)

    normalizer_assembly = xr.DataArray(rate,
                                       coords={'repetition': ('repetition', list(range(rate.shape[1]))),
                                               'stimulus_id': ('image', list(range(rate.shape[0]))),
                                               'id': ('image', list(range(rate.shape[0])))},
                                       dims=['image', 'repetition', 'neuroid'])
    for column_name, column_data in neuroid_meta.items():
        normalizer_assembly = normalizer_assembly.assign_coords(**{f'{column_name}': ('neuroid', list(column_data.values))})
    normalizer_assembly = normalizer_assembly.stack(presentation=('image', 'repetition')).reset_index('presentation')
    normalizer_assembly = normalizer_assembly.drop('image')
    normalizer_assembly = normalizer_assembly.transpose('presentation', 'neuroid')
    normalizer_assembly = NeuronRecordingAssembly(normalizer_assembly)

    print(normalizer_assembly)
    print(normalizer_assembly['stimulus_id'])
    #filtered_assembly = filter_neuroids(normalizer_assembly, 0.75)
    #assembly = assembly.sel(neuroid=np.isin(assembly.neuroid_id, filtered_assembly.neuroid_id))
    assembly = assembly.transpose('presentation', 'neuroid', 'time_bin')

    # Add other experiment info
    assembly.attrs['image_size_degree'] = 8
    assembly.attrs['stim_on_time_ms'] = 100

    return assembly